# Google Drive Ownership Transfer Bot

# Initialization

In [ ]:
# This is the TRANSFER BOT (Bot 1)
# Logs in using an existing Chrome session, opens a shared folder, recursively traverses folders,
# and transfers ownership of all files the user owns to a specified Gmail.

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# === CONFIGURATION ===
SHARED_FOLDER_URL = "https://drive.google.com/drive/folders/YOUR_SHARED_FOLDER_ID"
NEW_OWNER_EMAIL = "new.owner@gmail.com"
CHROME_PROFILE_PATH = "user-data-dir=/path/to/your/first/account/profile"

# === SETUP DRIVER ===
options = webdriver.ChromeOptions()
options.add_argument(CHROME_PROFILE_PATH)
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

visited_folders = set()

# === HELPER FUNCTIONS ===
def scroll_to_load_items():
    scroll_area = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]')))
    last_height = driver.execute_script("return arguments[0].scrollHeight", scroll_area)
    while True:
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll_area)
        time.sleep(2)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scroll_area)
        if new_height == last_height:
            break
        last_height = new_height

def transfer_ownership():
    file_elements = driver.find_elements(By.CSS_SELECTOR, 'div[aria-label][role="gridcell"]')
    print(f"Found {len(file_elements)} items")

    for file in file_elements:
        try:
            file_name = file.get_attribute("aria-label")
            if not file_name:
                continue
            file.click()
            time.sleep(1)

            # Open Share dialog
            webdriver.ActionChains(driver).key_down(Keys.CONTROL).key_down(Keys.SHIFT).send_keys('s').key_up(Keys.SHIFT).key_up(Keys.CONTROL).perform()
            time.sleep(3)

            try:
                owner_badge = wait.until(EC.presence_of_element_located((By.XPATH, '//span[text()="Owner"]')))
            except:
                print(f"Skipping '{file_name}': not owned by current user")
                webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                time.sleep(1)
                continue

            email_input = wait.until(EC.presence_of_element_located((By.XPATH, '//input[@type="email"]')))
            email_input.send_keys(NEW_OWNER_EMAIL)
            email_input.send_keys(Keys.RETURN)
            time.sleep(3)

            editor_dropdown = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[contains(text(),"Editor") or contains(text(),"Viewer") or contains(text(),"Commenter")]')))
            editor_dropdown.click()
            time.sleep(1)

            make_owner_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[text()="Make owner"]')))
            make_owner_button.click()
            time.sleep(1)

            send_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[text()="Send"]')))
            send_button.click()
            time.sleep(5)

        except Exception as e:
            print("Error processing file:", e)
            continue

def get_folder_links():
    links = []
    folders = driver.find_elements(By.XPATH, '//div[@aria-label][@role="gridcell" and descendant::div[@data-tooltip="Folder"]]')
    for folder in folders:
        try:
            folder_name = folder.get_attribute("aria-label")
            if folder_name in visited_folders:
                continue
            folder.click()
            time.sleep(3)
            current_url = driver.current_url
            visited_folders.add(folder_name)
            links.append(current_url)
            driver.execute_script("window.history.go(-1)")
            time.sleep(3)
        except:
            continue
    return links

def crawl_and_transfer(folder_url):
    driver.get(folder_url)
    time.sleep(3)
    scroll_to_load_items()
    transfer_ownership()
    folder_links = get_folder_links()
    for link in folder_links:
        crawl_and_transfer(link)

# === EXECUTION ===
crawl_and_transfer(SHARED_FOLDER_URL)
driver.quit()

In [ ]:
# This is the ACCEPT BOT (Bot 2)
# Logs into the recipient Gmail account, opens Gmail inbox, and accepts ownership transfer requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# === CONFIGURATION ===
GMAIL_URL = "https://mail.google.com"
CHROME_PROFILE_PATH = "user-data-dir=/path/to/your/second/account/profile"

# === SETUP DRIVER ===
options = webdriver.ChromeOptions()
options.add_argument(CHROME_PROFILE_PATH)
driver = webdriver.Chrome(options=options)
driver.get(GMAIL_URL)
wait = WebDriverWait(driver, 30)

# === WAIT FOR GMAIL TO LOAD ===
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.T-I.T-I-KE.L3')))
print("Gmail loaded.")

# === SEARCH FOR OWNERSHIP EMAILS ===
search_box = wait.until(EC.presence_of_element_located((By.NAME, 'q')))
search_box.clear()
search_box.send_keys("ownership transfer")
search_box.send_keys(Keys.RETURN)
time.sleep(5)

# === CLICK EACH OWNERSHIP EMAIL AND ACCEPT ===
emails = driver.find_elements(By.XPATH, '//span[@email and contains(text(),"drive-shares-noreply@google.com")]')
print(f"Found {len(emails)} transfer emails")

for i in range(len(emails)):
    try:
        emails[i].click()
        time.sleep(5)

        # Click the "Accept ownership" link
        accept_link = wait.until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, "accept ownership")))
        accept_link.click()
        time.sleep(5)

        # Go back to Gmail
        driver.get(GMAIL_URL)
        time.sleep(5)
    except Exception as e:
        print("Error accepting transfer:", e)
        driver.get(GMAIL_URL)
        time.sleep(5)
        continue

print("Done accepting all ownership transfers.")
driver.quit()

# Trials

# Testing

# End